In [1]:
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, GPT2Config

import torch
from torch.utils.data import Dataset
from tqdm import tqdm
from pathlib import Path
import numpy as np
from PIL import Image
import os
import re
import random

In [9]:
HEADER = '<?xml version="1.0" encoding="UTF-8"?> <?xml-model href="http://music-encoding.org/schema/4.0.0/mei-all.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?> <?xml-model href="http://music-encoding.org/schema/4.0.0/mei-all.rng" type="application/xml" schematypens="http://purl.oclc.org/dsdl/schematron"?> <mei xmlns="http://www.music-encoding.org/ns/mei" meiversion="4.0.0"> <meiHead> <fileDesc> <titleStmt> <title /> </titleStmt> <pubStmt> <date>2017-08-29 23:54:28</date> </pubStmt> </fileDesc> <encodingDesc> <projectDesc> <p>Encoded with Verovio version 2.0.0-dev-cba6710</p>  </projectDesc> </encodingDesc> </meiHead>'
FOOTER = '</mei>'

In [10]:
KEYSIGS = [
    'key.sig= "1f" ',
    'key.sig= "5f" ',
    'key.sig= "1s" ',
    'key.sig= "4s" ',
    'key.sig= "2s" ',
    'key.sig= "7s" ',
    'key.sig= "3s" ',
    'key.sig= "4f" ',
    'key.sig= "6s" ',
    '',
    'key.sig= "6f" ',
    'key.sig= "5s" ',
    'key.sig= "2f" ',
    'key.sig= "3f" '
]

CLEFS = [
    'clef.shape= "G" clef.line= "2"',
    'clef.shape= "C" clef.line= "5"',
    'clef.shape= "C" clef.line= "2"',
    'clef.shape= "F" clef.line= "4"',
    'clef.shape= "C" clef.line= "1"',
    'clef.shape= "C" clef.line= "4"',
    'clef.shape= "C" clef.line= "3"',
    'clef.shape= "F" clef.line= "3"',
    'clef.shape= "F" clef.line= "5"',
    'clef.shape= "G" clef.line= "1"'
]

TIMESIGS = [
    'meter.count= "2" meter.unit= "4"',
    'meter.count= "3" meter.unit= "4"',
    'meter.sym= "common"',
    'meter.count= "5" meter.unit= "4"',
    'meter.count= "6" meter.unit= "4"',
    'meter.count= "7" meter.unit= "4"',
    'meter.count= "3" meter.unit= "8"',
    'meter.count= "6" meter.unit= "8"',
]

def generate_randomized_mei_seed(timesig, start_token="<s>"):
    keysig = KEYSIGS[int(random.random() * len(KEYSIGS))]
    clef = CLEFS[int(random.random() * len(CLEFS))]
    return f'{start_token} <music > <body > <mdiv > <score > <scoreDef {keysig}{timesig} > <staffGrp > <staffDef {clef}'

In [14]:
TIMESIGS_READABLE = [
    '2-4',
    '3-4',
    'C',
    '5-4',
    '6-4',
    '7-4',
    '3-8',
    '6-8'
]

In [12]:
VOCAB_SIZE = 30000
MAX_LEN = 512

In [13]:
TOKENIZER_SAVEDIR = Path('/home/macosta/ttmp/primus-data/primus-mei/mei-tokenizer/')
LM_MODEL_SAVEDIR = Path('/home/macosta/ttmp/primus-models/gpt2-lm-mei/')
PRIMUS_TXT_FILES = Path('/home/macosta/ttmp/primus-data/primus-mei/mei/')

In [14]:
def load_model():
    config = GPT2Config(
        vocab_size=VOCAB_SIZE,
        n_positions=MAX_LEN,
        n_head=12,
    )
    model = GPT2LMHeadModel(config=config).from_pretrained(str(LM_MODEL_SAVEDIR))
    return model

In [6]:
model = load_model()

In [7]:
tokenizer = GPT2TokenizerFast.from_pretrained(TOKENIZER_SAVEDIR, max_len=MAX_LEN)

file /home/macosta/ttmp/primus-data/primus-mei/mei-tokenizer/config.json not found
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
def unclean(mei):
    mei = re.sub('<s>', '', mei)
    mei = re.sub('</s>', '', mei)
    mei = re.sub(' (/?>)', r'\1', mei)
    mei = re.sub('= ', '=', mei)
    mei = HEADER + mei + FOOTER
    return mei

In [16]:
def test_engravability(n, write_dir):
    write_dir.mkdir(exist_ok=True)
    commands = ''
    for i in tqdm(range(n)):
        input_str = tokenizer.encode('<s>', return_tensors="pt")
        output_tokens = model.generate(input_str, 
                                       pad_token_id=1,
                                       eos_token_id=2,
                                       temperature=1,
                                       max_length=512,
                                       do_sample=True)[0]
        output_tokens = tokenizer.decode(output_tokens).split()
        mei = unclean(' '.join(output_tokens[1:-1]))
        writepath = write_dir / f'generated_mei_{i}.mei'
        with open(writepath, 'w') as f:
            f.write(mei)
        commands += f'verovio {writepath} -o {str(writepath)[:-4]}.svg 2> {str(writepath)[:-4]}_output.txt\n'
    with open(write_dir / 'engrave.sh', 'w') as f:
        f.write(commands)

In [10]:
def create_engrave_script_for_subdirs(directories):
    commands = ''
    for directory in directories:
        for file in os.listdir(directory):
            if file[-4:] != '.mei':
                continue
            full_filename = str(directory / file)
            commands += f'verovio {full_filename} -o {full_filename[:-4]}.svg 2> {full_filename[:-4]}_output.txt\n'
    with open('engrave.sh', 'w') as f:
        f.write(commands)

In [32]:
def create_engrave_script(directory):
    commands = ''
    for file in os.listdir(directory):
        if file[-4:] != '.mei':
            continue
        full_filename = directory / file
        no_extension = str(full_filename)[:-4]
#         log_file = directory / f"{no_extension.split('/')[-1]}-output.txt"
        log_file = f"/home/macosta/ttmp/irmak_outputs/{no_extension.split('/')[-1]}-output.txt"
#         commands += f'verovio {full_filename} -o {no_extension}.svg 2> {log_file}\n'
        commands += f'verovio {full_filename} -o /dev/null/ 2> {log_file}\n'
    with open('engrave.sh', 'w') as f:
        f.write(commands)

In [55]:
def save_timesig_generated(n, timesig, savedir):
    savedir.mkdir(exist_ok=True, parents=True)
    for i in tqdm(range(n)):
        seed = generate_randomized_mei_seed(timesig)
        input_ids = tokenizer.encode(seed, return_tensors='pt')
        output_tokens = model.generate(input_ids, 
                                       pad_token_id=1,
                                       eos_token_id=2,
                                       temperature=1,
                                       max_length=512,
                                       do_sample=True)[0]
        output_tokens = tokenizer.decode(output_tokens).split()
        with open(savedir / f"generated_{i}.mei", "w") as f:
            f.write(unclean(' '.join(output_tokens)))

In [17]:
def count_warnings_and_errors(output_files):
    warnings = 0
    unique_warnings = set()
    failed = ''
    for output_file in output_files:
        with open(output_file, "r") as f:
            contents = f.read()
        if 'Warning' in contents or 'Error' in contents:
            warnings += 1
            failed += str(output_file).split('-output.txt')[0] + '.mei\n'
        content_lines = contents.split('\n')
        for content_line in content_lines:
            if 'Warning' in content_line:
                unique_warnings.add(content_line)
    print(f"{warnings}/{len(output_files)}")
    with open('failed_engravings.txt', 'a') as f:
        f.write(failed)
    return unique_warnings

In [15]:
for t in TIMESIGS_READABLE:
    create_engrave_script(Path(f'/home/macosta/ttmp/generated-mei-timesigs-final/{t}/'))

In [33]:
create_engrave_script(Path(f'/home/ibukey/ttmp/mei-uncleaned/'))

In [11]:
# directories = [Path(f"/home/macosta/ttmp/generated-mei-timesigs/{timesig.replace('/','-')}/") for timesig in TIMESIGS]
# create_engrave_script_for_subdirs(directories)

In [ ]:
# write_dir = Path('/home/macosta/ttmp/generated_mei_sanity/')
# testing_harness(1000, write_dir)

In [46]:
# create_engrave_script(Path('/home/macosta/ttmp/generated-mei-timesigs/6-4/'))

In [35]:
# output_files = [list(Path(write_dir).glob("**/*_output.txt")) for write_dir in directories]
output_files = list(Path('/home/macosta/ttmp/irmak_outputs/').glob("**/*-output.txt"))
unique_warnings = count_warnings_and_errors(output_files)

129/15029


In [56]:
N = 2000
for i in range(len(TIMESIGS)):
    readable_timesig = TIMESIGS_READABLE[i]
    timesig = TIMESIGS[i]
    save_timesig_generated(N, timesig, Path(f'/home/macosta/ttmp/generated-mei-timesigs-final/{readable_timesig}/'))

  0%|                                                                                                                                 | 0/10 [00:00<?, ?it/s]

0


 10%|████████████                                                                                                             | 1/10 [00:04<00:42,  4.71s/it]

1


 20%|████████████████████████▏                                                                                                | 2/10 [00:10<00:41,  5.22s/it]

...

1


 20%|████████████████████████▏                                                                                                | 2/10 [00:10<00:42,  5.33s/it]

2


 20%|████████████████████████▏                                                                                                | 2/10 [00:11<00:44,  5.52s/it]


KeyboardInterrupt: 